In [1]:
import numpy as np
from collections import OrderedDict
from scipy.spatial import distance as dist
import cv2

In [2]:
class Tracker():
    def __init__(self, max_disappear=50):
        self.nextObjectID = 0
        self.objects = OrderedDict() ### Ordered dictionary (ids and last location)
        self.disappeared = OrderedDict() ## Track which id disappeared for how many frames
        self.maxDisappeared = max_disappear ## Threshold to delete the id
        
    def register(self, centroid): ## Method to add the new object(id)
        self.objects[self.nextObjectID] = centroid ## Add to the objects dict 
        self.disappeared[self.nextObjectID] = 0 ##
        self.nextObjectID += 1 
    
    def deregister(self, objectID):
        del self.objects[objectID]
        del self.disappeared[objectID]
        
    def update(self, bbox):
        if len(bbox) == 0:
            for objectID in self.objects.keys():
                self.disappeared[objectID] += 1
                if self.disappeared[objectID] > self.maxDisappeared:
                    self.deregister(objectID)
            return self.objects
        
        inputCentroids = np.zeros((len(bbox), 2), dtype=np.int32)
        for i, rec in enumerate(bbox):
            x,y,w,h = rec
            cx = x + int(w/2)
            cy = y + int(h/2)
            inputCentroids[i] = [cx,cy]
        
        if len(self.objects) == 0:  ### 1st time -- tracking -- 1st frame
            for i in range(len(bbox)):
                self.register(inputCentroids[i])
            return self.objects
        
        objectIDS = list(self.objects.keys())
        objectCentroids = list(self.objects.values())
        
        dist_mat = dist.cdist(np.array(objectCentroids), inputCentroids)
        
        tracker_id = dist_mat.argmin(axis=1).argsort() ### for each id finding the bbox closet to it
        
        observed_id = dist_mat.argmin(axis=1)[tracker_id] ## each of the ids-- the bbox matchings (if 2 bobes pt to same centroid so finding the nearest box for that centroid)
        
        used_tracked = set()
        used_observed = set()
        
        for (track, obser) in zip(tracker_id, observed_id):
            if (track in used_tracked or obser in used_observed) or dist_mat[track, obser] > 50:
                continue
                
            objectID = objectIDS[track]
            self.objects[objectID] = inputCentroids[obser]
            self.disappeared[objectID] = 0
            
            used_tracked.add(track)
            used_observed.add(obser)
            
        unused_tracked = set(range(0, dist_mat.shape[0])).difference(used_tracked)
        unused_observed = set(range(0, dist_mat.shape[0])).difference(used_observed)
        
        if dist_mat.shape[0] >= dist_mat.shape[1] : ## ids > bboxes
            for row_id in unused_tracked:
                objectID = objectIDS[row_id]
                self.disappeared[objectID] += 1
                
                if self.disappeared[objectID] > self.maxDisappeared:
                    self.deregister(objectID)
                    
        else:
            for col_id in unused_observed:
                self.register(inputCentroids[col_id])
        
        return self.objects

In [ ]:
cap = cv2.VideoCapture("videoplayback.mp4")

tracker = Tracker()

if cap.isOpened() == False:
    print("Error in Video")
    run = False
else:
    run = True
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
while run:
    
    ret, frame = cap.read()
    
    if ret:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        detected_faces = face_cascade.detectMultiScale(gray, 1.2, 10)
        
        for [x, y, w, h] in detected_faces:
            cv2.rectangle(frame, (x,y), (x+w, y+h), (255,0,0), 4)
            
        tracking_info = tracker.update(detected_faces)
        
        for (tracking_id, tracking_cent) in tracking_info.items():
            text = f"ID:{tracking_id}"
            print(tracking_cent)
            cv2.putText(frame, text, (tracking_cent[0]-10, tracking_cent[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0),2)
            cv2.circle(frame, (tracking_cent[0], tracking_cent[1]), 4, (0,255,0), -1)
            
            cv2.imshow("Tracker", frame)
            
            if cv2.waitKey(25) & 0xff == ord("q"):
                break
    else:
        break

[439  63]
[439  60]
[439  60]
[439  60]
[439  60]
[439  60]
[439  60]
[439  60]
[442  50]
[443  52]
[443  52]
[443  52]
[443  52]
[443  52]
[443  52]
[443  52]
[443  52]
[443  52]
[443  52]
[443  52]
[443  52]
[443  52]
[443  52]
[443  52]
[443  52]
[443  52]
[443  52]
[443  52]
[443  52]
[443  52]
[443  52]
[443  52]
[430  48]
[430  48]
[430  48]
[430  48]
[430  48]
[430  48]
[430  48]
[430  48]
[430  48]
[430  48]
[430  48]
[430  48]
[430  48]
[430  48]
[430  48]
[430  48]
[430  48]
[430  48]
[430  48]
[430  48]
[430  48]
[430  48]
[430  48]
[430  48]
[430  48]
[430  48]
[430  48]
[430  48]
[430  48]
[430  48]
[430  48]
[430  48]
[430  48]
[430  48]
[430  48]
[430  48]
[430  48]
[430  48]
[430  48]
[430  48]
[430  48]
[430  48]
[430  48]
[451  86]
[450  85]
[448  86]
[448  86]
[445  82]
[443  81]
[443  81]
[441  78]
[441  77]
[440  77]
[440  77]
[440  77]
[425  77]
[420  78]
[415  78]
[417  80]
[417  79]
[417  80]
[416  80]
[417  81]
[417  81]
[417  81]
[417  81]
[417  81]
[417  81]
